In [1]:
!pip install -q langchain langchain-community langchain-google-genai pypdf requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires googl

In [2]:
import os
from google.colab import files, userdata

from langchain_community.document_loaders import (
    TextLoader,
    PyPDFLoader,
    CSVLoader
)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

GEMINI_API_KEY = userdata.get("Assignments")

In [3]:
print("=" * 60)
print("📂 UNIVERSAL FILE SUMMARIZER")
print("=" * 60)
print("Supported formats: .txt, .pdf, .csv\n")
uploaded = files.upload()
if not uploaded:
    raise ValueError("❌ No file uploaded.")

file_name = list(uploaded.keys())[0]
print(f"\n✅ Uploaded file: {file_name}")
if file_name.endswith(".txt"):
    print("📄 Detected TXT file")
    loader = TextLoader(file_name, encoding="utf-8")

elif file_name.endswith(".pdf"):
    print("📕 Detected PDF file")
    loader = PyPDFLoader(file_name)

elif file_name.endswith(".csv"):
    print("📊 Detected CSV file")
    loader = CSVLoader(file_name)

else:
    raise ValueError("❌ Unsupported file type")

docs = loader.load()
print("\n✅ Content loaded successfully!")
print(f"• Type of docs: {type(docs)}")
print(f"• Number of documents/pages: {len(docs)}")

total_chars = sum(len(doc.page_content) for doc in docs)
print(f"• Total characters: {total_chars:,}")

preview_len = 300
print("\n📋 Content Preview:")
print("-" * 60)
print(docs[0].page_content[:preview_len])
if len(docs[0].page_content) > preview_len:
    print("...")
print("-" * 60)
combined_content = "\n\n".join(doc.page_content for doc in docs)

max_chars = 15000
if len(combined_content) > max_chars:
    print(f"\n⚠️ Truncating content to {max_chars:,} characters")
    combined_content = combined_content[:max_chars] + "\n\n[Content truncated]"
print("\n" + "-" * 60)
print("🤖 Generating summary using Gemini")
print("-" * 60)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.7,
    google_api_key=GEMINI_API_KEY
)

prompt = PromptTemplate(
    input_variables=["content"],
    template="""
You are an expert summarizer.

Create a clear 10-point bullet summary of the content below.
Each bullet point should:
- Capture a key insight
- Be concise and informative
- Be a complete sentence
- Cover a different aspect of the content

Content:
{content}

Return exactly 10 bullet points.
"""
)

chain = prompt | llm | StrOutputParser()
print("⏳ Summarizing content...")

try:
    summary = chain.invoke({"content": combined_content})

    print("\n" + "=" * 60)
    print("📝 SUMMARY")
    print("=" * 60)
    print(summary)
    print("\n✅ Summarization complete!")

except Exception as e:
    print(f"\n❌ Error: {e}")
save_choice = input("\n💾 Save summary to file? (yes/no): ").lower()

if save_choice == "yes":
    output_file = f"summary_{file_name.rsplit('.', 1)[0]}.txt"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(f"SUMMARY OF: {file_name}\n")
        f.write("=" * 60 + "\n\n")
        f.write(summary)

    print(f"✅ Saved as {output_file}")
    files.download(output_file)

📂 UNIVERSAL FILE SUMMARIZER
Supported formats: .txt, .pdf, .csv



Saving adnan_backend_resume.pdf to adnan_backend_resume.pdf

✅ Uploaded file: adnan_backend_resume.pdf
📕 Detected PDF file

✅ Content loaded successfully!
• Type of docs: <class 'list'>
• Number of documents/pages: 1
• Total characters: 1,976

📋 Content Preview:
------------------------------------------------------------
Adnan Irshad Bagwan 
Pune, Maharashtra 
   9373952126 | ✉ adnan.bagwan.cse@gmail.com |   Linkedin| Github  
Professional Summary 
Backend Developer with hands-on experience in designing and implementing scalable server-side applications. Skilled in API 
development, third-party API integration, and 
...
------------------------------------------------------------

------------------------------------------------------------
🤖 Generating summary using Gemini
------------------------------------------------------------
⏳ Summarizing content...

📝 SUMMARY
Here is a 10-point bullet summary of the provided content:

*   Adnan Irshad Bagwan is a Backend Developer with exper

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>